In [1]:
import tensorflow as tf
c = tf.constant("Hello, distributed TensorFlow!")
server = tf.train.Server.create_local_server() #Creates a server
sess = tf.Session(server.target)  # Create a session on the server.
sess.run(c)

b'Hello, distributed TensorFlow!'

In [2]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
server = tf.train.Server(cluster, job_name="local", task_index=0)
#cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
#server = tf.train.Server(cluster, job_name="local", task_index=1)

In [5]:
cluster.as_dict()

{'local': ['localhost:2222', 'localhost:2223']}

In [1]:
from tensorflow.python.client import device_lib
print("*******Device Info Seen By TensorFlow*******")
print(device_lib.list_local_devices())

*******Device Info Seen By TensorFlow*******
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14646601112833075701
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 15917685146
locality {
  bus_id: 2
}
incarnation: 2329635678512156560
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:81:00.0"
]


In [2]:
import tensorflow as tf
with tf.device('/device:GPU:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
with tf.device("/job:ps/task:0"):
    weights_1 = tf.Variable(...)
    biases_1 = tf.Variable(...)

with tf.device("/job:ps/task:1"):
    weights_2 = tf.Variable(...)
    biases_2 = tf.Variable(...)

with tf.device("/job:worker/task:7"):
    input, labels = ...
    layer_1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    logits = tf.nn.relu(tf.matmul(layer_1, weights_2) + biases_2)
  # ...
    train_op = ...

with tf.Session("grpc://worker7.example.com:2222") as sess:
    for _ in range(10000):
        sess.run(train_op)

array([[22., 28.],
       [49., 64.]], dtype=float32)

In [7]:
#Distributed TensorFlow : Cluster Creation Example

cluster = tf.train.ClusterSpec({"worker": [
        "gpu047.pvt.bridges.psc.edu:2222",
        "gpu046.pvt.bridges.psc.edu:2222"
    ]#,
    #"ps": [
    #    "localhost:2222",
    #]
                               })

In [17]:
server = tf.train.Server(cluster, job_name="worker", task_index=0)
server = tf.train.Server(cluster, job_name="worker", task_index=1)
#server_3 = tf.train.Server(cluster, job_name="ps", task_index=0)

In [1]:
import tensorflow as tf
with tf.device("/cpu:0"):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
with tf.device("/gpu:0"):
    c = tf.matmul(a, b)
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [13]:
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1259741900768258021
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 605290496
locality {
  bus_id: 2
}
incarnation: 17834486825444952031
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:81:00.0"
]


In [1]:
# Creates a graph.
import tensorflow as tf
c = []
for d in ['/device:GPU:2', '/device:GPU:3']:
    with tf.device(d):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
        c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
    sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(sum))

[[ 44.  56.]
 [ 98. 128.]]


In [1]:
import tensorflow as tf
tf.__version__

'1.0.1'

In [2]:
cluster_spec = tf.train.ClusterSpec({'ps' : ['localhost:2222'], 
                                     #'worker' : ['localhost:2223'] 
                                    })

In [3]:
server = tf.train.Server(cluster_spec,job_name='ps',task_index=0)

In [4]:
server.target

b'grpc://localhost:2222'

In [5]:
%%bash
lsof -i -P -n | grep LISTEN | grep python

python    10287 asakhare   12u  IPv4 63058391      0t0  TCP 127.0.0.1:42232 (LISTEN)
python    10287 asakhare   14u  IPv4 63058393      0t0  TCP 127.0.0.1:52143 (LISTEN)
python    10287 asakhare   16u  IPv4 63058395      0t0  TCP 127.0.0.1:41934 (LISTEN)
python    10287 asakhare   18u  IPv4 63058397      0t0  TCP 127.0.0.1:52705 (LISTEN)
python    10287 asakhare   22u  IPv4 63058400      0t0  TCP 127.0.0.1:37493 (LISTEN)
python    10287 asakhare   31u  IPv4 63043182      0t0  TCP 127.0.0.1:35252 (LISTEN)
python    10287 asakhare   65u  IPv6 63053177      0t0  TCP *:2222 (LISTEN)
python    10287 asakhare   66u  IPv6 63053178      0t0  TCP *:2222 (LISTEN)


In [10]:
server.server_def

cluster {
  job {
    name: "local"
    tasks {
      value: "localhost:0"
    }
  }
}
job_name: "local"
protocol: "grpc"

In [11]:
sess = tf.Session(target=server.target)

In [12]:
server = tf.train.Server.create_local_server()

In [13]:
%%bash
lsof -i -P -n | grep LISTEN | grep python

python    10287 asakhare   12u  IPv4 63058391      0t0  TCP 127.0.0.1:42232 (LISTEN)
python    10287 asakhare   14u  IPv4 63058393      0t0  TCP 127.0.0.1:52143 (LISTEN)
python    10287 asakhare   16u  IPv4 63058395      0t0  TCP 127.0.0.1:41934 (LISTEN)
python    10287 asakhare   18u  IPv4 63058397      0t0  TCP 127.0.0.1:52705 (LISTEN)
python    10287 asakhare   22u  IPv4 63058400      0t0  TCP 127.0.0.1:37493 (LISTEN)
python    10287 asakhare   31u  IPv4 63043182      0t0  TCP 127.0.0.1:35252 (LISTEN)
python    10287 asakhare   65u  IPv6 63053177      0t0  TCP *:2222 (LISTEN)
python    10287 asakhare   66u  IPv6 63053178      0t0  TCP *:2222 (LISTEN)
python    10287 asakhare   74u  IPv6 63059510      0t0  TCP *:38480 (LISTEN)
python    10287 asakhare   75u  IPv6 63059511      0t0  TCP *:33208 (LISTEN)
python    10287 asakhare   82u  IPv6 63043190      0t0  TCP *:36788 (LISTEN)
python    10287 asakhare   83u  IPv6 63043191      0t0  TCP *:44686 (LISTEN)


In [22]:
#works above tf 1.4
#devices = sess.list_devices()
#for d in devices:
#    print(d.name)

#/job:worker/replica:0/task:0/device:CPU:0

In [ ]:
import tensorflow as tf

In [ ]:
cluster = tf.train.ClusterSpec({
    "worker": [
        "localhost:2222",
        "localhost:2223",
    ],
    "ps": [
        "localhost:2224",
        "localhost:2225"
    ]})

In [1]:
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']

'0'

In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [9]:
os.environ['CUDA_VISIBLE_DEVICES']

''

In [3]:
type_='ps'

In [4]:
if (type_=='ps'):
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
else:
    pass

In [5]:
os.environ['CUDA_VISIBLE_DEVICES']

''

In [6]:
from tensorflow.python.client import device_lib
print("*******Device Info Seen By TensorFlow*******")
print(device_lib.list_local_devices())

*******Device Info Seen By TensorFlow*******
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13016095190179405664
]
